<font face="Arial" size="6" color="white"> **ProfessionAI** |Master AI Development: Programmazione con Python| </font></br>[[filippo gronchi](mailto:fgronchi&#64;gmail.com) @ babbage25]


---



# <a tag="title">Final Project - Un Algoritmo di Correzione per un Motore di Ricerca</a>

### <font face="Arial" size="5" color="white"><u>Table of contents</u>
</font><a class='anchor' name='toc'></a>
- [1. Purpose](#1)
- [2. Edit distance](#2)
    - [2.1 Levenshtein distance](#2.1)
- [3. Implementation steps](#3)
- [4. Code](#4)
    - [4.1 Word Dictionary Creation](#4.1)
    - [4.2 Levenshtein Distance function](#4.2)
    - [4.3 Suggest Correction function](#4.3)
- [5. Verification](#5)
    - [5.1 Test Environment](#5.1)
    - [5.2 Static Test](#5.2)
    - [5.3 Interactive Test](#5.3)

## <a name="1">1. Purpose</a>

>Python implementation of a function **suggest_correction(query, dictionary)** to detect and correct typos as input to a custom search system. This function compares the input string (consisting of one or more words) with all the words in its internal dictionary. It returns as output a string composed of the same number of words. This string will be identical to the input if no typos are detected. Otherwise, words with one or more typos will be replaced by those in the dictionary whose edit distance is minimal.

[$\uparrow$ top](#toc)

## <a name="2">2. Edit distance</a>


Edit distance (or editing distance) between two strings represents the minimum number of elementary operations required to transform one string into the other. The operations usually considered are: insertion of a character, deletion of a character and substitution of a character.

Each of these operations, for example, can have the same 'cost', and the edit distance corresponds to the minimum total number of these operations.

There are several algorithms to compute this distance: *Levenshtein*, *Hamming*, *Damerau-Levenshtein*, *Longest Common Subsequences (LCS)*, etc.

### <a name="2.1">2.1 Levenshtein Distance</a>

For the purposes of this project, the most appropriate algorithm is [**Levenshtein Distance**](https://it.wikipedia.org/wiki/Distanza_di_Levenshtein), as it can also detect cases where characters are added to or removed from words therefore can compare strings of different lengths. Levenshtein distance between 2 words is equal to the minimum number of manipulations needed to obtain the second starting from the first. Transformations to be considered are:
* Adding a character
* Deleting a character
* Replacing a character

[$\uparrow$ top](#toc)

## <a name="3">3. Implementation Steps</a>

* Create a "dictionary" of 50 common italian words to be used as golden refence.
> To this aim the file *parole.txt* from the GitHub repository [napolux/paroleitaliane](https://github.com/napolux/paroleitaliane) is used. 50 words  are then sampled from it (equally distributed in the list). The package *request* and its method *get* will be used to upload the dictionary file directly from Github while the package *pprint* will provide a better values display.
```python
requests.get(<filename.txt>)
```

* Code of [Levenshtein Distance](#2.1) algorythm following *ProfAI* suggested approach and encapsulate it in a dedicated function.
```python
function levenshtein_dist(a_word, b_word)
```

* Code the main function, taking 2 strings ar arguments and returning a string (Correct Query) and an integer (Edit Distance).
```python
function suggest_correction(query, dictionary)
```

* Code the verification function, taking 2 strings, result dataframe and integer test number and returning a string with the result and the dataframe updated.
```python
function TestQuery (query, dictionary, df, test_num)
```

[$\uparrow$ top](#toc)

## <a name="4">4. Code</a>

### <a name="4.1">4.1 Word Dictionary creation</a>

In [ ]:
# input file for word dictionary from REPO napolux/paroleitaliane
word_file_raw = "https://raw.githubusercontent.com/napolux/paroleitaliane/refs/heads/main/paroleitaliane/1000_parole_italiane_comuni.txt"

# we connect directly to the repo using get method
# the object downloaded is of str type
import requests
response = requests.get(word_file_raw)
file_content = response.text

# master dictionary building sampling 50 equally distributed words from the original file
word_list_span = round(len(file_content.split())/50,0)
original_word_list = file_content.split()

word_list = []
for word in original_word_list:
  if not original_word_list.index(word)%word_list_span:
     word_list.append(word)
  if len(word_list) == 50:
    break

# Display the full dictionary to be used
import pprint
pprint.pprint(word_list)

['a',
 'aiutare',
 'anima',
 'aspetto',
 'avanti',
 'bestia',
 'campo',
 'certamente',
 'collina',
 'concedere',
 'contrario',
 'cristiano',
 'di',
 'disposizione',
 'dubbio',
 'errore',
 'fame',
 'finalmente',
 'fratello',
 'gioia',
 'greco',
 'improvviso',
 'interesse',
 'lavoro',
 'luce',
 'mare',
 'mettere',
 'morte',
 'nessuno',
 'occhio',
 'orecchio',
 'particolare',
 'pericoloso',
 'più',
 'potere',
 'privato',
 'pubblicare',
 'raccogliere',
 'ricco',
 'risposta',
 'saltare',
 'secolo',
 'sforzo',
 'solito',
 'spesso',
 'strano',
 'tanto',
 'tornare',
 'un',
 'vero']


### <a name="4.2">4.2 Levenshtein Distance function</a>

The following implementation of Levenshtein distance calculation has been suggested by ProfAI. The code fits the project needs since it's easy to read/understand and quick enough to execute.

In [ ]:

def levenshtein(w1, w2):
  """
  function to evaluate the levenshtein distance between 2 strings
  input: w1 -> str, w2 -> str
  output: levenshtein distance -> int
  """
  m, n = len(w1), len(w2)
  # define new matrix (m+1) x (n+1)
  dp = [[0 for _ in range(n+1)] for _ in range(m+1)]

  # Init first row, first column
  for i in range(m+1):
    dp[i][0] = i   # all characters from w1 in w2 (all deletion)
  for j in range(n+1):
    dp[0][j] = j   # all characters from w2 in w1 (all addition)

  # fill up the matrix
  for i in range(1, m+1):
    for j in range(1, n+1):
      if w1[i-1] == w2[j-1]:
        cost = 0 # in case no substitution needed
      else:
        cost = 1
      dp[i][j] = min(
        dp[i-1][j] + 1,    # deletion
        dp[i][j-1] + 1,    # addition
        dp[i-1][j-1] + cost  # substitution
      )

  return dp[m][n]

### <a name="4.3">4.3 Suggest Correction function</a>

Main function according to the Project specification.

In [ ]:
def suggest_correction(query, dictionary):
  """
  function to analyze the input string and provide the closer one considering
  the dictionary. All the characters are considered lowercase
  input: query -> str, dictionary -> list of str
  output: correct query -> str,
          total edit distance from query and correct query -> int
  """
  # internal variable initialization
  edit_dist_total = 0
  suggested_query = []
  query_list = query.lower().split()

  # Max Word Length in Dictionary
  # this is to initializa the max distance at
  # the beginning of each evaluation
  max_length_word = 0
  for w in dictionary:
    if len(w) > max_length_word:
      max_length_word = len(w)

  # Evaluate min edit distance and corresponding dictionary word
  # cycle variable initialization
  edit_dist_total = 0
  correct_query = []
  for word_q in query_list:
    # check first above all dictionary words
    # if present edit distance doesn't need to be
    # calculated
    if word_q in dictionary:
      min_dist = 0
      correct_word = word_q
    else:
      # init word-to-word edit distance with max possible values
      min_dist = max(len(word_q),max_length_word)
      for word_d in dictionary:
        if levenshtein(word_q, word_d) < min_dist:
          min_dist = levenshtein(word_q, word_d)
          correct_word = word_d
    edit_dist_total += min_dist
    correct_query.append(correct_word)
  return " ".join(correct_query), edit_dist_total

[$\uparrow$ top](#toc)

## <a name="5">5. Verification</a>

Test strategy is divided in 2 parts:
* Static test: where several situations (positive and negative) will be covered statically. At the end a pandas dataframe will show a recap of all the runs
* Interactive test: the reader will be able to enter a query and to receive the correct version of it along the Edit Distance from the Dictionary

### <a name="5.1">5.1 Test environment

The following function will be used to run static test cases and provide a nice looking output.

In [ ]:
def TestQuery (query, dictionary, df, test_num):
  """
  Function to be used for testing. Execute the suggest_correction function and
  return in text format the entered query, correct query and edit distance.
  Finally the recap df gets updated appending a new line.
  input: query -> str, dictionary -> list of str, df -> pandas dataframe
  output: str, df
  """
  correct_query, edit_dist = suggest_correction(query,dictionary)
  result = "Entered query: ====> "+query+"\nCorrect query: ====> "+correct_query+"\nEdit distance = "+str(edit_dist)
  new_row = {'[test #]': test_num, '[query]': query, '[correct query]': correct_query, '[edit distance]': str(edit_dist)}
  new_row_df = pd.DataFrame([new_row])
  df = pd.concat([df, new_row_df], ignore_index=True)
  return result, df

Result dataframe creation (empty).

In [ ]:
import pandas as pd
# create the empty dataframe
column_name = ["[test #]", "[query]", "[correct query]", "[edit distance]"]
result_df = pd.DataFrame(columns=column_name)

### <a name="5.2">5.2 Static Test</a>

A list of 10 queries are used to run the full regression test.

In [ ]:
# test query list
test_query_list = [
    "rispost particulari e pericolose",
    "orecchio",
    "soliti secol al lavoro",
    "più potere a disposizione",
    "occhio particolare da solito un secoli",
    "tanto strano",
    "Aiutare A Saltare",
    "Finalmente una gioia",
    "Nessu i partiolare  epericoloso tornar stranissimo",
    "ricco aspetto saltare sforzo spesso collina a contrario"
]

Regression test will run checking all the elements of the test_query_list.

In [ ]:
test_num = 1
for query in test_query_list:
  print(f"Static Test {test_num}\n==============")
  result, result_df = TestQuery (query, word_list, result_df, test_num)
  print(result+"\n")
  test_num += 1

print("\nStatic test summary:\n====================\n\n"+result_df.to_string(index=False))

Static Test 1
Entered query: ====> rispost particulari e pericolose
Correct query: ====> risposta particolare a pericoloso
Edit distance = 5

Static Test 2
Entered query: ====> orecchio
Correct query: ====> orecchio
Edit distance = 0

Static Test 3
Entered query: ====> soliti secol al lavoro
Correct query: ====> solito secolo a lavoro
Edit distance = 3

Static Test 4
Entered query: ====> più potere a disposizione
Correct query: ====> più potere a disposizione
Edit distance = 0

Static Test 5
Entered query: ====> occhio particolare da solito un secoli
Correct query: ====> occhio particolare a solito un secolo
Edit distance = 2

Static Test 6
Entered query: ====> tanto strano
Correct query: ====> tanto strano
Edit distance = 0

Static Test 7
Entered query: ====> Aiutare A Saltare
Correct query: ====> aiutare a saltare
Edit distance = 0

Static Test 8
Entered query: ====> Finalmente una gioia
Correct query: ====> finalmente un gioia
Edit distance = 1

Static Test 9
Entered query: ====> Ne

### <a name="5.3">5.3 Interactve Test</a>

User will be able to enter personal query and verify the function outcome. The cycle will exit when 'quit' is typed.

In [ ]:
result_df = pd.DataFrame(columns=column_name)
test_num = 1
test_input = input("Enter a new query (type quit to exit): ")
while test_input!="quit":
  result, result_df = TestQuery (test_input, word_list, result_df, test_num)
  print(result)
  test_num += 1
  test_input = input("\nEnter a new query (type quit to exit): ")

print("\n\nInteractive test summary:\n=========================\n\n"+result_df.to_string(index=False))

Enter a new query (type quit to exit): ciao allora
Entered query: ====> ciao allora
Correct query: ====> a anima
Edit distance = 7

Enter a new query (type quit to exit): benissimo
Entered query: ====> benissimo
Correct query: ====> bestia
Edit distance = 5

Enter a new query (type quit to exit): ottimo a bestia
Entered query: ====> ottimo a bestia
Correct query: ====> anima a bestia
Edit distance = 4

Enter a new query (type quit to exit): quit


Interactive test summary:

[test #]         [query] [correct query] [edit distance]
       1     ciao allora         a anima               7
       2       benissimo          bestia               5
       3 ottimo a bestia  anima a bestia               4


[$\uparrow$ top](#toc)



---

